# Проект: ВШЭ ПСБ.Хак

## Описание задачи


После суровых трудовых будней всегда хочется с удовольствием провести свой отпуск и насладиться отдыхом. Тем не менее, на этом пути к счастью обычно возникают несколько преград: выбор желаемой гостиницы, заблаговременное бронирование, оплата, получение максимально возможной скидки и т.д. Как минимизировать свои риски и траты, и при этом получить желаемый уровень сервиса и отдых?

А что, если проанализировать данные по оплатам, бронированиям и отменам для сети гостиниц и попытаться найти скрытые зависимости с датами заезда, формой оплаты и другими признаками?

А если на этом не останавливаться и попытаться использовать найденные паттерны для создания нового банковского продукта, который сможет как помочь потенциальным клиентам с организацией отпуска, так и позволит банку и отелю остаться в плюсе?

Именно для создания такого инновационного продукта Вышка Онлайн совместно с Цифровой лабораторией ПСБ проводит хакатон ВШЭ ПСБ.Хак.


### Организация


Соревнование пройдет с 20-22 сентября 2024 в гибридном формате. Командам участникам необходимо пройти через два этапа:

* **Этап 1.** Технический. Решение задачи бинарной классификации с целью предсказания факта отмены бронирования
* **Этап 2.** Бизнесовый. На основе найденных зависимостей в рамках первого этапа, а также с учетом самостоятельно проведенного анализа рынка и существующих банковских и онлайн-туристических продуктов и программ лояльности, предложить новый банковский продукт для банка ПСБ.
В рамках первого этапа команды подгружают свои предсказания отмены бронирования на платформу, где происходит автоматическая проверка ответов на тестовой выборке. Первый этап соревнования завершается 22 сентября в 16:00.

Детали регистрации на платформе и особенности отправки решений будут подробно описаны после торжественного открытия хакатона 20 сентября.

В рамках второго этапа команды готовят презентацию предлагаемого продукта, которую будут защищать перед жюри 22 сентября, начиная с 17:00.

Для погружения участников в особенности банковского бизнеса и предметной области в течение субботы будет проводится серия мастер-классов. Отдельно с командами будут работать менторы, которые будут консультировать на протяжении всего периода соревнования.

### Данные
Данные представляют собой информацию о бронировании номеров различных категорий в разных гостиницах. Гостиницы с номерами 1 и 2 – один регион РФ, с номерами 3 и 4 – другой регион РФ. Структура данных следующая:



* № брони – идентификатор брони
* Номеров – количество номеров в бронировании
* Стоимость – стоимость номеров в рублях
* Внесена предоплата – сумма внесенной предоплаты
* Способ оплаты – один из 12 способов оплаты
* Дата бронирования – дата бронирования с точностью до минуты
* Дата отмены – дата отмены бронирования с точностью до минуты, если было
* Заезд – дата заезда с точностью до дня
* Ночей – количество ночей
* Выезд – дата выезда с точностью до дня
* Источник – онлайн-канал продаж
* Статус брони – один из 5 статусов
* Категория номера – описание категории номера. Если бронировалось несколько номеров, то идет сплошное описание с * нумерацией.
* Гостей – число гостей
* Гостиница – номер гостиницы

* Целевое поле – Дата отмены: если поле заполнено, то это соответствует целевому значению 1, иначе – 0. Статусы в поле Статус брони уточняют состояние и носят справочный характер (но могут помочь при построении модели).

В тестовой выборке отсутствуют столбцы Дата отмены и Статус брони.

### Метрика
Итоговой метрикой выступает ROC-AUC по столбцу Дата отмены.


### Лидерборд
Лидерборд технической части соревнования (метрика модели) в процессе соревнования считается по доступной для скачивания тестовой выборке. Окончание отправок всех решений 22 сентября в 16:00. Лидерборд скрывается от участников 22 сентября в 12:00 и открывается с итоговым результатом 22 сентября в 20:00.

### Итоговая оценка решения
Итоговая оценка соревнования формируется на основе технической части (40% результата) и защит решений в форме презентаций (60% результата). Экспертное жюри определяет оценку защит. Детали с критериями оценки второго этапа будут предоставлены командам отдельно.

## Загрузка данных

### Импорт библиотек

In [2]:
import pandas as pd
import requests
import re
from io import BytesIO
import os

### Функции

In [3]:
def github(url):
    response = requests.get(url)
    response.raise_for_status() 
    return BytesIO(response.content)

In [4]:
# Функция для загрузки файла из локальной папки или с GitHub
def load_f(file_path, github_url, file_type="xlsx"):
    try:
        # Сначала пытаемся загрузить из локальной папки
        if file_type == "xlsx":
            data = pd.read_excel(file_path, index_col=0)  # Устанавливаем первую колонку в качестве индекса
        elif file_type == "csv":
            data = pd.read_csv(file_path, index_col=0)  # Устанавливаем первую колонку в качестве индекса
        print(f"Файл загружен локально: {file_path}")
    except FileNotFoundError:
        # Если файл не найден, загружаем с GitHub
        print(f"Файл не найден локально, загружаю с GitHub: {github_url}")
        if file_type == "xlsx":
            data = pd.read_excel(download_file_from_github(github_url), index_col=0)  # Устанавливаем первую колонку в качестве индекса
        elif file_type == "csv":
            data = pd.read_csv(download_file_from_github(github_url), index_col=0)  # Устанавливаем первую колонку в качестве индекса
        print(f"Файл загружен с GitHub: {github_url}")
    return data

In [5]:
# Пути к локальным файлам и ссылки на GitHub
train_local_path = os.path.join('datasets', 'train.xlsx')
test_local_path = os.path.join('datasets', 'test.xlsx')
example_local_path = os.path.join('datasets', 'example.csv')

train_github_url = "https://github.com/EPrutskoy/MindGames_Hotel_Product/blob/3e23c7d7c49ec53e08e9e3d17468f26d286f6b4e/datasets/train.xlsx?raw=true"
test_github_url = "https://github.com/EPrutskoy/MindGames_Hotel_Product/blob/3e23c7d7c49ec53e08e9e3d17468f26d286f6b4e/datasets/test.xlsx?raw=true"
example_github_url = "https://github.com/EPrutskoy/MindGames_Hotel_Product/blob/3e23c7d7c49ec53e08e9e3d17468f26d286f6b4e/datasets/example.csv?raw=true"

# Загрузка файлов
train_data = load_f(train_local_path, train_github_url, file_type="xlsx")
test_data = load_f(test_local_path, test_github_url, file_type="xlsx")
example_data = load_f(example_local_path, example_github_url, file_type="csv")

Файл загружен локально: datasets\train.xlsx
Файл загружен локально: datasets\test.xlsx
Файл загружен локально: datasets\example.csv


### Загрузка таблиц

In [6]:
train_data.info()
test_data.info()
example_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26174 entries, 0 to 26173
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   № брони             26174 non-null  object        
 1   Номеров             26174 non-null  int64         
 2   Стоимость           26174 non-null  float64       
 3   Внесена предоплата  26174 non-null  int64         
 4   Способ оплаты       26174 non-null  object        
 5   Дата бронирования   26174 non-null  datetime64[ns]
 6   Дата отмены         5192 non-null   datetime64[ns]
 7   Заезд               26174 non-null  datetime64[ns]
 8   Ночей               26174 non-null  int64         
 9   Выезд               26174 non-null  datetime64[ns]
 10  Источник            26174 non-null  object        
 11  Статус брони        26174 non-null  object        
 12  Категория номера    26174 non-null  object        
 13  Гостей              26174 non-null  int64         


In [7]:
test_data.columns = [
    'booking_number', 
    'room_count', 
    'price', 
    'prepayment_amount',
    'payment_method', 
    'booking_date', 
    'check_in', 
    'nights', 
    'check_out',
    'source', 
    'room_category', 
    'guests', 
    'hotel']

train_data.columns = [
    'booking_number',        # № брони
    'room_count',            # Номеров
    'price',                 # Стоимость
    'prepayment_amount',     # Внесена предоплата
    'payment_method',        # Способ оплаты
    'booking_date',          # Дата бронирования
    'cancellation_date',     # Дата отмены
    'check_in',              # Заезд
    'nights',                # Ночей
    'check_out',             # Выезд
    'source',                # Источник
    'booking_status',        # Статус брони
    'room_category',         # Категория номера
    'guests',                # Гостей
    'hotel'                  # Гостиница
]

# Проверим, что столбцы изменились
print(train_data.columns)
print(test_data.columns)

Index(['booking_number', 'room_count', 'price', 'prepayment_amount',
       'payment_method', 'booking_date', 'cancellation_date', 'check_in',
       'nights', 'check_out', 'source', 'booking_status', 'room_category',
       'guests', 'hotel'],
      dtype='object')
Index(['booking_number', 'room_count', 'price', 'prepayment_amount',
       'payment_method', 'booking_date', 'check_in', 'nights', 'check_out',
       'source', 'room_category', 'guests', 'hotel'],
      dtype='object')


### Объединение таблиц

In [8]:
# Добавление столбца 'is_canceled'
train_data['is_canceled'] = train_data['cancellation_date'].apply(lambda x: 1 if pd.notnull(x) else 0)


In [9]:
# Используем concat для объединения двух таблиц, даже если у них разные столбцы
combined_data = pd.concat([train_data, test_data], axis=0, join='outer', ignore_index=True)


In [10]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37392 entries, 0 to 37391
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   booking_number     37392 non-null  object        
 1   room_count         37392 non-null  int64         
 2   price              37392 non-null  float64       
 3   prepayment_amount  37392 non-null  int64         
 4   payment_method     37392 non-null  object        
 5   booking_date       37392 non-null  datetime64[ns]
 6   cancellation_date  5192 non-null   datetime64[ns]
 7   check_in           37392 non-null  datetime64[ns]
 8   nights             37392 non-null  int64         
 9   check_out          37392 non-null  datetime64[ns]
 10  source             37392 non-null  object        
 11  booking_status     26174 non-null  object        
 12  room_category      37392 non-null  object        
 13  guests             37392 non-null  int64         
 14  hotel 

In [11]:
# Проверим результат
display(combined_data.head())

,booking_number,room_count,price,prepayment_amount,payment_method,booking_date,cancellation_date,check_in,nights,check_out,source,booking_status,room_category,guests,hotel,is_canceled
0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1,1.0
1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1,0.0
2,20221204-16563-171020423,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4,0.0
3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3,0.0
4,20230529-6634-200121971,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1,0.0


### Таблица `combined_data`

In [12]:
# Функция для разделения информации
def extract_payment_info(payment_method):
    # Устанавливаем начальные значения по умолчанию
    payment_type = "Нет"
    provider = "Нет"
    special_conditions = "Нет"
    prepayment = 0
    
    # Проверка способа оплаты
    if 'Банк. карта' in payment_method:
        payment_type = 'Банк. Карта'
    elif 'Отложенная электронная оплата' in payment_method:
        payment_type = 'Отложенная электронная оплата'
    elif 'Система быстрых платежей' in payment_method:
        payment_type = 'СБП'
    elif 'Внешняя система оплаты' in payment_method:
        payment_type = 'Внешняя система оплаты'
    elif 'При заселении' in payment_method:
        payment_type = 'При заселении'

    # Проверка провайдера (банка)
    provider_match = re.search(r'Банк\s([^\(]+)', payment_method)
    cashback_match = re.search(r'Кешбэк\. МИР', payment_method)
    if provider_match:
        provider = provider_match.group(1).strip()
    elif cashback_match:
        provider = 'Кешбэк. МИР'
    
    # Проверка на эквайринг или кешбэк в условиях
    if 'Эквайринг' in payment_method:
        special_conditions = re.search(r'Эквайринг\s([^\(]+)', payment_method).group(0)
    elif 'Кешбэк' in payment_method:
        special_conditions = 'Кешбэк'
    
    # Проверка на предоплату
    if 'С предоплатой' in payment_method:
        prepayment = 1

    return payment_type, provider, special_conditions, prepayment


In [13]:
def replace_exact_room_type(df, column_name):
    # Заменяем строки, которые равны "Номер", на "Номер Стандарт"
    df[column_name] = df[column_name].apply(lambda x: 'Номер "Стандарт"' if x == 'Номер' else x)
    return df


In [14]:
# Функция для очистки столбца от дубликатов
def clean_source(source):
    # Удаляем данные в скобках, если они есть, так как они могут содержать дополнительные даты или данные
    source_cleaned = re.sub(r"\s*\(.*?\)", "", source).strip()
    return source_cleaned


In [15]:
def categorize_source(source):
    if source in ['Официальный сайт', 'Программа лояльности', 'Бронирование из экстранета']:
        return 'Собственные каналы бронирования'
    elif source in ['booking.com', 'expedia.com', 'ostrovok.ru', 'OneTwoTrip', 'Bronevik.com', 
                    'Bronevik.com/Bro.Online', '101hotels.com', 'Cuva','Яндекс.Путешествия']:
        return 'Крупные международные платформы'
    elif source in ['Alean.ru', 'Acase.ru', 'Zabroniryi.ru']:
        return 'Партнерские системы бронирования'
    elif source in ['ВКонтакте', 'Otello', 'Тинькофф Путешествия', 'Ozon', 'Svoy Hotel']:
        return 'Социальные сети и локальные сервисы'
    else:
        return 'Другое'

In [16]:
# Функция для обработки room_category
def process_room_category(room_category):
    # Разделяем строку по разделителю '\n'
    rooms = room_category.split('\n')
    
    # Первое название номера — это первый элемент списка
    first_room = rooms[0].split(' ')[1] if '.' in rooms[0] else rooms[0]
    
    return first_room.strip()

In [17]:
def normalize_and_remove_duplicates(df, column_name):
    # Нормализуем категории в столбце
    df[column_name] = df[column_name].apply(lambda x: 
                                            'Апартаменты' if 'Апартаменты' in x else 
                                            'Коттедж' if 'Коттедж' in x else 
                                            'Студия' if 'Студия' in x else 
                                            'Стандарт' if 'Стандарт' in x else 
                                            'Люкс' if 'Люкс' in x else x)
    
    
    return df


In [23]:
def assign_region(hotel):
    if hotel in [1, 2]:
        return 1
    elif hotel in [3, 4]:
        return 2
    else:
        return None

In [61]:
# Пайплайн обработки данных
def process_data(df):
    # 1) room_count - все значения больше 3 приравнять к 3
    df['room_count'] = df['room_count'].apply(lambda x: min(x, 3))
    
    # 2) price - все значения выше 175 000 приравнять к 175 000
    df['price'] = df['price'].apply(lambda x: min(x, 108000))
    
    # 3) prepayment_amount - все значения выше 90 000 приравнять к 90 000
    df['prepayment_amount'] = df['prepayment_amount'].apply(lambda x: min(x, 90000))
    
    # 4) payment_method - используем функцию extract_payment_info
    payment_columns = ['payment_method', 'payment_provider', 'payment_conduction', 'is_prepayment']
    df[payment_columns] = df['payment_method'].apply(lambda x: pd.Series(extract_payment_info(x)))
    
    # 5) Даты (booking_date, cancellation_date, check_in, check_out) оставляем без изменений
    
    # 6) nights - значения больше 5 приравнять к 5
    df['nights'] = df['nights'].apply(lambda x: min(x, 5))
    df['guests'] = df['guests'].apply(lambda x: min(x, 6))
    # 7) source - используем функцию clean_source
    df['source'] = df['source'].apply(clean_source)
    
    # 8) booking_status - оставляем без изменений
    # Применяем функцию для столбца source и создаем новый столбец category
    df['source_category'] = df['source'].apply(categorize_source)

    # Применяем функцию к столбцу room_category и создаем два новых столбца
    df[['room_category']] = df['room_category'].apply(lambda x: pd.Series(process_room_category(x)))

# Пример использования:
    df = replace_exact_room_type(df, 'room_category')

    df = normalize_and_remove_duplicates(df, 'room_category')

# Применяем функцию к столбцу hotel
    df['region'] = df['hotel'].apply(assign_region)


    # Преобразуем строки с датами в формат datetime
    df['booking_date'] = pd.to_datetime(df['booking_date'])
    df['check_in'] = pd.to_datetime(df['check_in'])

# 1. Добавляем новый столбец с разницей в днях между датой брони и заезда
    df['days_before_checkin'] = (df['check_in'] - df['booking_date']).dt.days
    df['days_before_checkin'] = df['days_before_checkin'].apply(lambda x: min(x, 180))
# 2. Добавляем столбец с средней стоимостью за ночь
    df['average_price_per_night'] = df['price'] / df['nights']
# Добавляем столбец с ценой за одну ночь на одного человека
    df['price_per_night_per_guest'] = df['average_price_per_night'] / df['guests']
# Проверяем результат
    return df

In [62]:
# Применение пайплайна к train_data
processed_data = process_data(combined_data)

In [63]:
processed_data.sample(5)

,booking_number,room_count,price,prepayment_amount,payment_method,booking_date,cancellation_date,check_in,nights,check_out,...,hotel,is_canceled,payment_provider,payment_conduction,is_prepayment,source_category,region,days_before_checkin,average_price_per_night,price_per_night_per_guest
7834,20220220-6634-120949872,1,15296.0,15296,Нет,2022-01-14 10:29:00,NaT,2022-02-20 15:00:00,2,2022-02-22 12:00:00,...,1,0.0,Нет,Нет,0,Собственные каналы бронирования,1,37,7648.0,3824.0
10618,20231104-16563-218699068,2,72200.0,72200,Нет,2023-08-16 08:55:28,NaT,2023-11-04 15:00:00,2,2023-11-06 12:00:00,...,4,0.0,Нет,Нет,0,Собственные каналы бронирования,2,80,36100.0,9025.0
36916,20230122-6634-179694829,1,6300.0,6300,Нет,2023-01-19 14:16:10,NaT,2023-01-22 15:00:00,1,2023-01-23 12:00:00,...,1,NaN,Нет,Нет,0,Собственные каналы бронирования,1,3,6300.0,6300.0
30148,20230718-7491-209388048,1,28800.0,14400,Нет,2023-07-05 23:26:39,NaT,2023-07-18 15:00:00,2,2023-07-20 12:00:00,...,3,NaN,Нет,Нет,0,Собственные каналы бронирования,2,12,14400.0,7200.0
27353,20220812-7492-126555862,1,28400.0,14200,Отложенная электронная оплата,2022-02-28 11:09:15,NaT,2022-08-12 15:00:00,2,2022-08-14 12:00:00,...,2,NaN,Нет,Нет,0,Собственные каналы бронирования,1,165,14200.0,7100.0


In [64]:
display(processed_data.groupby(['hotel', 'room_category']).size().reset_index(name='counts'))


,hotel,room_category,counts
0,1,Апартаменты,2857
1,1,Коттедж,2003
2,1,Люкс,1097
3,1,Стандарт,6159
4,2,Стандарт,6850
5,2,Студия,3754
6,3,Стандарт,11370
7,4,Студия,3302


In [65]:
processed_data.info()
processed_data.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37392 entries, 0 to 37391
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   booking_number             37392 non-null  object        
 1   room_count                 37392 non-null  int64         
 2   price                      37392 non-null  float64       
 3   prepayment_amount          37392 non-null  int64         
 4   payment_method             37392 non-null  object        
 5   booking_date               37392 non-null  datetime64[ns]
 6   cancellation_date          5192 non-null   datetime64[ns]
 7   check_in                   37392 non-null  datetime64[ns]
 8   nights                     37392 non-null  int64         
 9   check_out                  37392 non-null  datetime64[ns]
 10  source                     37392 non-null  object        
 11  booking_status             26174 non-null  object        
 12  room

,booking_number,room_count,price,prepayment_amount,payment_method,booking_date,cancellation_date,check_in,nights,check_out,...,hotel,is_canceled,payment_provider,payment_conduction,is_prepayment,source_category,region,days_before_checkin,average_price_per_night,price_per_night_per_guest
13220,20220521-7491-136064108,1,9600.0,0,Отложенная электронная оплата,2022-04-28 11:32:02,2022-05-05 11:33:47,2022-05-21 15:00:00,1,2022-05-22 12:00:00,...,3,1.0,Нет,Нет,0,Собственные каналы бронирования,2,23,9600.0,4800.0


In [66]:
# Просмотр первых строк с новыми столбцами
display(processed_data.head())

,booking_number,room_count,price,prepayment_amount,payment_method,booking_date,cancellation_date,check_in,nights,check_out,...,hotel,is_canceled,payment_provider,payment_conduction,is_prepayment,source_category,region,days_before_checkin,average_price_per_night,price_per_night_per_guest
0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,...,1,1.0,Нет,Нет,0,Крупные международные платформы,1,7,8566.666667,4283.333333
1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,...,1,0.0,Нет,Нет,0,Собственные каналы бронирования,1,23,12400.000000,6200.000000
2,20221204-16563-171020423,1,25800.0,12900,Нет,2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,...,4,0.0,Нет,Нет,0,Собственные каналы бронирования,2,19,12900.000000,6450.000000
3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты,2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,...,3,0.0,Нет,Нет,0,Крупные международные платформы,2,9,10500.000000,10500.000000
4,20230529-6634-200121971,1,28690.0,28690,Нет,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,...,1,0.0,Нет,Нет,0,Собственные каналы бронирования,1,8,14345.000000,3586.250000


In [70]:
# Группируем данные по региону, отелю, категории номера и вычисляем среднюю цену на одного гостя за ночь
grouped_data = processed_data.groupby(['region', 'hotel', 'room_category']).agg({
    'price_per_night_per_guest': 'mean'
}).reset_index()

# Переименовываем столбец для ясности
grouped_data.rename(columns={'price_per_night_per_guest': 'avg_price_per_night_per_guest'}, inplace=True)

# Просмотр первых строк сгруппированных данных
display(grouped_data)

,region,hotel,room_category,avg_price_per_night_per_guest
0,1,1,Апартаменты,6044.944494
1,1,1,Коттедж,10723.954332
2,1,1,Люкс,6078.453682
3,1,1,Стандарт,5338.308358
4,1,2,Стандарт,5894.402407
5,1,2,Студия,7752.834181
6,2,3,Стандарт,5318.527640
7,2,4,Студия,9069.823529


In [68]:
processed_data['average_price_per_night'].mean()

15627.403471776493

In [71]:
# Убедимся, что колонка 'check_in' в формате даты
processed_data['check_in'] = pd.to_datetime(processed_data['check_in'])

# Фильтруем данные только за 2022 год
data_2022 = processed_data[processed_data['check_in'].dt.year == 2022]

# Группируем данные по региону и считаем количество заездов
checkins_2022 = data_2022.groupby('region').size().reset_index(name='checkin_count')

# Просмотр результатов
print(checkins_2022)

   region  checkin_count
0       1          10680
1       2           7328


In [72]:
# Убедимся, что колонка 'check_in' в формате даты
processed_data['check_in'] = pd.to_datetime(processed_data['check_in'])

# Фильтруем данные только за 2022 год
data_2022 = processed_data[processed_data['check_in'].dt.year == 2022]

# Группируем данные по региону и отелю, считая количество заездов
checkins_2022 = data_2022.groupby(['region', 'hotel']).size().reset_index(name='checkin_count')

# Просмотр результатов
print(checkins_2022)

   region  hotel  checkin_count
0       1      1           5506
1       1      2           5174
2       2      3           5726
3       2      4           1602


In [79]:
# Создание нового столбца guests_in_room
processed_data['guests_in_room'] = processed_data['guests'] / processed_data['room_count']

# Просмотр первых строк с новым столбцом
print(processed_data[['guests', 'room_count', 'guests_in_room']].head())

   guests  room_count  guests_in_room
0       2           1             2.0
1       2           1             2.0
2       2           1             2.0
3       1           1             1.0
4       4           1             4.0


In [80]:
# Создание столбца с месяцем заезда
processed_data['month'] = processed_data['check_in'].dt.month

# Функция для определения сезона
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

# Создание столбца с сезоном заезда
processed_data['season'] = processed_data['month'].apply(get_season)

# Просмотр первых строк с новыми столбцами
print(processed_data[['check_in', 'month', 'season']].head())

             check_in  month  season
0 2023-04-28 15:00:00      4  Spring
1 2022-07-11 15:00:00      7  Summer
2 2022-12-04 15:00:00     12  Winter
3 2023-09-18 15:00:00      9  Autumn
4 2023-05-29 15:00:00      5  Spring


In [108]:
# Пример данных о заказах processed_data, где есть столбцы с месяцем и сезоном
# Пример данных о вместимости отелей
hotel_capacity = pd.DataFrame({
    'region': [1, 1, 1, 1, 1, 1, 2, 2],
    'hotel': [1, 1, 1, 1, 2, 2, 3, 4],
    'room_category': ['Апартаменты', 'Коттедж', 'Люкс', 'Стандарт', 'Стандарт', 'Студия', 'Стандарт', 'Студия'],
    'num_rooms': [20, 26, 5, 42, 31, 13, 61, 15],
    'capacity_per_room': [5, 6, 3, 2, 2, 4, 3, 4]
})

# Расчет полной вместимости каждого отеля по каждой категории номера
hotel_capacity['total_capacity'] = hotel_capacity['num_rooms'] * hotel_capacity['capacity_per_room']


In [109]:
hotel_capacity

,region,hotel,room_category,num_rooms,capacity_per_room,total_capacity
0,1,1,Апартаменты,20,5,100
1,1,1,Коттедж,26,6,156
2,1,1,Люкс,5,3,15
3,1,1,Стандарт,42,2,84
4,1,2,Стандарт,31,2,62
5,1,2,Студия,13,4,52
6,2,3,Стандарт,61,3,183
7,2,4,Студия,15,4,60


In [110]:

# Группировка по отелю и сезону, чтобы посчитать общее количество гостей
guests_per_season = processed_data.groupby(['region', 'hotel', 'season']).agg({
    'guests_in_room': 'sum'  # Количество гостей в каждом сезоне
}).reset_index()

# Присоединяем данные о вместимости отелей
hotel_season_capacity = pd.merge(guests_per_season, hotel_capacity, on=['region', 'hotel'], how='left')


In [111]:
hotel_season_capacity

,region,hotel,season,guests_in_room,room_category,num_rooms,capacity_per_room,total_capacity
0,1,1,Autumn,6909.833333,Апартаменты,20,5,100
1,1,1,Autumn,6909.833333,Коттедж,26,6,156
2,1,1,Autumn,6909.833333,Люкс,5,3,15
3,1,1,Autumn,6909.833333,Стандарт,42,2,84
4,1,1,Spring,7524.000000,Апартаменты,20,5,100
5,1,1,Spring,7524.000000,Коттедж,26,6,156
6,1,1,Spring,7524.000000,Люкс,5,3,15
7,1,1,Spring,7524.000000,Стандарт,42,2,84
8,1,1,Summer,9179.333333,Апартаменты,20,5,100
9,1,1,Summer,9179.333333,Коттедж,26,6,156


In [112]:
# Группируем данные по отелю, сезону и категории номера, суммируя количество комнат
rooms_per_season = processed_data.groupby(['hotel', 'season', 'room_category']).agg({
    'room_count': 'sum'  # Суммируем количество комнат
}).reset_index()

# Переименуем колонку для ясности
rooms_per_season.rename(columns={'room_count': 'rooms_occupied'}, inplace=True)

# Просмотр результатов
print(rooms_per_season)

    hotel  season room_category  rooms_occupied
0       1  Autumn   Апартаменты             592
1       1  Autumn       Коттедж             444
2       1  Autumn          Люкс             279
3       1  Autumn      Стандарт            1142
4       1  Spring   Апартаменты             634
5       1  Spring       Коттедж             445
6       1  Spring          Люкс             245
7       1  Spring      Стандарт            1438
8       1  Summer   Апартаменты             822
9       1  Summer       Коттедж             512
10      1  Summer          Люкс             304
11      1  Summer      Стандарт            1894
12      1  Winter   Апартаменты             843
13      1  Winter       Коттедж             643
14      1  Winter          Люкс             269
15      1  Winter      Стандарт            1988
16      2  Autumn      Стандарт            1861
17      2  Autumn        Студия            1033
18      2  Spring      Стандарт            1824
19      2  Spring        Студия         

In [113]:
hotel_season_capacity['rooms_occupied'] = rooms_per_season['rooms_occupied']

In [116]:
hotel_season_capacity[['region', 'hotel', 'season', 'room_category','rooms_occupied', 'guests_in_room']]

,region,hotel,season,room_category,rooms_occupied,guests_in_room
0,1,1,Autumn,Апартаменты,592,6909.833333
1,1,1,Autumn,Коттедж,444,6909.833333
2,1,1,Autumn,Люкс,279,6909.833333
3,1,1,Autumn,Стандарт,1142,6909.833333
4,1,1,Spring,Апартаменты,634,7524.000000
5,1,1,Spring,Коттедж,445,7524.000000
6,1,1,Spring,Люкс,245,7524.000000
7,1,1,Spring,Стандарт,1438,7524.000000
8,1,1,Summer,Апартаменты,822,9179.333333
9,1,1,Summer,Коттедж,512,9179.333333


In [118]:

# Рассчитываем процент заполненности
hotel_season_capacity['occupancy_rate'] = (hotel_season_capacity['guests_in_room'] / hotel_season_capacity['rooms_occupied']) 

# Просмотр первых строк с результатами
display(hotel_season_capacity[['region', 'hotel', 'season', 'room_category', 'occupancy_rate']])

,region,hotel,season,room_category,occupancy_rate
0,1,1,Autumn,Апартаменты,11.672016
1,1,1,Autumn,Коттедж,15.562688
2,1,1,Autumn,Люкс,24.766428
3,1,1,Autumn,Стандарт,6.050642
4,1,1,Spring,Апартаменты,11.867508
5,1,1,Spring,Коттедж,16.907865
6,1,1,Spring,Люкс,30.710204
7,1,1,Spring,Стандарт,5.232267
8,1,1,Summer,Апартаменты,11.167072
9,1,1,Summer,Коттедж,17.928385


In [122]:
# Выполняем слияние
processed_data = processed_data.merge(
    hotel_season_capacity[['region', 'hotel', 'season', 'room_category', 'occupancy_rate']],
    on=['region', 'hotel', 'season', 'room_category'],
    how='left'
)

# Если создаются дублирующие столбцы, оставляем только один
processed_data.drop(columns=['occupancy_rate_x'], inplace=True)  # Удаляем лишний столбец
processed_data.rename(columns={'occupancy_rate_y': 'occupancy_rate'}, inplace=True)  # Переименовываем для ясности

# Просмотр обновленного DataFrame
display(processed_data.head())

,booking_number,room_count,price,prepayment_amount,payment_method,booking_date,cancellation_date,check_in,nights,check_out,...,is_prepayment,source_category,region,days_before_checkin,average_price_per_night,price_per_night_per_guest,guests_in_room,month,season,occupancy_rate
0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,...,0,Крупные международные платформы,1,7,8566.666667,4283.333333,2.0,4,Spring,5.232267
1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,...,0,Собственные каналы бронирования,1,23,12400.000000,6200.000000,2.0,7,Summer,4.846533
2,20221204-16563-171020423,1,25800.0,12900,Нет,2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,...,0,Собственные каналы бронирования,2,19,12900.000000,6450.000000,2.0,12,Winter,2.240496
3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты,2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,...,0,Крупные международные платформы,2,9,10500.000000,10500.000000,1.0,9,Autumn,1.925586
4,20230529-6634-200121971,1,28690.0,28690,Нет,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,...,0,Собственные каналы бронирования,1,8,14345.000000,3586.250000,4.0,5,Spring,30.710204


In [123]:
processed_data

,booking_number,room_count,price,prepayment_amount,payment_method,booking_date,cancellation_date,check_in,nights,check_out,...,is_prepayment,source_category,region,days_before_checkin,average_price_per_night,price_per_night_per_guest,guests_in_room,month,season,occupancy_rate
0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,...,0,Крупные международные платформы,1,7,8566.666667,4283.333333,2.0,4,Spring,5.232267
1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,...,0,Собственные каналы бронирования,1,23,12400.000000,6200.000000,2.0,7,Summer,4.846533
2,20221204-16563-171020423,1,25800.0,12900,Нет,2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,...,0,Собственные каналы бронирования,2,19,12900.000000,6450.000000,2.0,12,Winter,2.240496
3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты,2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,...,0,Крупные международные платформы,2,9,10500.000000,10500.000000,1.0,9,Autumn,1.925586
4,20230529-6634-200121971,1,28690.0,28690,Нет,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,...,0,Собственные каналы бронирования,1,8,14345.000000,3586.250000,4.0,5,Spring,30.710204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37387,20220507-7492-130458541,1,50200.0,50200,Нет,2022-03-27 21:30:38,NaT,2022-05-07 15:00:00,2,2022-05-09 12:00:00,...,0,Собственные каналы бронирования,1,40,25100.000000,6275.000000,4.0,5,Spring,5.968202
37388,20240217-6634-235901857,1,108000.0,43500,Нет,2023-11-11 17:51:55,NaT,2024-02-16 15:00:00,5,2024-02-21 12:00:00,...,0,Собственные каналы бронирования,1,96,21600.000000,5400.000000,4.0,2,Winter,15.636858
37389,20220209-6634-124107676,1,42300.0,42300,Нет,2022-02-09 02:23:14,NaT,2022-02-09 15:00:00,1,2022-02-10 12:00:00,...,0,Собственные каналы бронирования,1,0,42300.000000,8460.000000,5.0,2,Winter,15.636858
37390,20230128-6634-179977236,1,27900.0,27900,Нет,2023-01-21 09:10:17,NaT,2023-01-28 15:00:00,1,2023-01-29 12:00:00,...,0,Собственные каналы бронирования,1,7,27900.000000,6975.000000,4.0,1,Winter,11.927046


In [134]:
# Сохранение DataFrame processed_data в CSV файл
processed_data.to_csv('processed_data.csv', index=False)

# Этот код сохранит файл в текущую директорию, где запущен Jupyter Notebook.
# После этого вы сможете загрузить файл через интерфейс Jupyter или из локальной системы.

In [126]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37392 entries, 0 to 37391
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   booking_number             37392 non-null  object        
 1   room_count                 37392 non-null  int64         
 2   price                      37392 non-null  float64       
 3   prepayment_amount          37392 non-null  int64         
 4   payment_method             37392 non-null  object        
 5   booking_date               37392 non-null  datetime64[ns]
 6   cancellation_date          5192 non-null   datetime64[ns]
 7   check_in                   37392 non-null  datetime64[ns]
 8   nights                     37392 non-null  int64         
 9   check_out                  37392 non-null  datetime64[ns]
 10  source                     37392 non-null  object        
 11  booking_status             26174 non-null  object        
 12  room

In [128]:
# Train выборка: строки, где is_canceled не является NaN
train = processed_data[processed_data['is_canceled'].notna()]

# Test выборка: строки, где is_canceled является NaN
test = processed_data[processed_data['is_canceled'].isna()]

# Убираем ненужные столбцы из тестовой выборки
test = test.drop(columns=['cancellation_date', 'booking_status', 'is_canceled'])

# Проверим размеры полученных выборок
print("Размер тренировочной выборки:", train.shape)
print("Размер тестовой выборки:", test.shape)

# Вывод первых строк выборок для проверки
display(train.head())
display(test.head())

Размер тренировочной выборки: (26174, 28)
Размер тестовой выборки: (11218, 25)


,booking_number,room_count,price,prepayment_amount,payment_method,booking_date,cancellation_date,check_in,nights,check_out,...,is_prepayment,source_category,region,days_before_checkin,average_price_per_night,price_per_night_per_guest,guests_in_room,month,season,occupancy_rate
0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,...,0,Крупные международные платформы,1,7,8566.666667,4283.333333,2.0,4,Spring,5.232267
1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,...,0,Собственные каналы бронирования,1,23,12400.000000,6200.000000,2.0,7,Summer,4.846533
2,20221204-16563-171020423,1,25800.0,12900,Нет,2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,...,0,Собственные каналы бронирования,2,19,12900.000000,6450.000000,2.0,12,Winter,2.240496
3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты,2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,...,0,Крупные международные платформы,2,9,10500.000000,10500.000000,1.0,9,Autumn,1.925586
4,20230529-6634-200121971,1,28690.0,28690,Нет,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,...,0,Собственные каналы бронирования,1,8,14345.000000,3586.250000,4.0,5,Spring,30.710204


,booking_number,room_count,price,prepayment_amount,payment_method,booking_date,check_in,nights,check_out,source,...,is_prepayment,source_category,region,days_before_checkin,average_price_per_night,price_per_night_per_guest,guests_in_room,month,season,occupancy_rate
26174,20231129-16563-238946689,1,23750.0,23750,Нет,2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,...,0,Собственные каналы бронирования,2,1,11875.000000,3958.333333,3.0,11,Autumn,2.200228
26175,20221219-7491-174959103,1,15010.0,7505,Нет,2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,...,0,Собственные каналы бронирования,2,6,7505.000000,3752.500000,2.0,12,Winter,1.893933
26176,20221211-6634-172724329,1,8400.0,8400,Отложенная электронная оплата,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,...,0,Собственные каналы бронирования,1,15,8400.000000,4200.000000,2.0,12,Winter,5.057596
26177,20230821-6634-212247350,1,42500.0,42500,Нет,2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,...,0,Собственные каналы бронирования,1,33,14166.666667,3541.666667,4.0,8,Summer,11.167072
26178,20230326-6634-189784563,1,62500.0,11900,Нет,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,...,0,Собственные каналы бронирования,1,3,12500.000000,12500.000000,1.0,3,Spring,5.232267


In [129]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26174 entries, 0 to 26173
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   booking_number             26174 non-null  object        
 1   room_count                 26174 non-null  int64         
 2   price                      26174 non-null  float64       
 3   prepayment_amount          26174 non-null  int64         
 4   payment_method             26174 non-null  object        
 5   booking_date               26174 non-null  datetime64[ns]
 6   cancellation_date          5192 non-null   datetime64[ns]
 7   check_in                   26174 non-null  datetime64[ns]
 8   nights                     26174 non-null  int64         
 9   check_out                  26174 non-null  datetime64[ns]
 10  source                     26174 non-null  object        
 11  booking_status             26174 non-null  object        
 12  room_cate

In [132]:
# Сохранение DataFrame processed_data в CSV файл
train.to_csv('train.csv', index=False)

In [133]:
test.to_csv('test.csv', index=False)